In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
headlines = pd.read_csv('./Datasets/Headlines/News_headlines.csv', parse_dates=['Date'])
headlines.head()

,Date,Headlines
0,2015-01-01,What Can We Expect From Apple Inc. In 2015? 1 ...
1,2015-02-01,What to expect from Apple in 2015 beyond its s...
2,2015-03-01,Mountie: An inexpensive and innovative way to ...
3,2015-04-01,"Donald Yacktman on the Sources of Moats, His C..."
4,2015-05-01,"SIM-free iPhone 6, 6 Plus reportedly debuting ..."


In [32]:
apple_data = pd.read_csv('./Datasets/Historical/HistoricalData_APPLE.csv', parse_dates=['Date'])
apple_data.head()

,Date,Close/Last,Volume,Open,High,Low
0,2021-06-09,$127.13,56877940,$127.21,$127.75,$126.52
1,2021-06-08,$126.74,74403770,$126.6,$128.46,$126.2101
2,2021-06-07,$125.9,71057550,$126.17,$126.32,$124.8321
3,2021-06-04,$125.89,75169340,$124.07,$126.16,$123.85
4,2021-06-03,$123.54,76229170,$124.68,$124.85,$123.13


In [33]:
apple_data.sort_values('Date',inplace=True)
apple_data.head()

,Date,Close/Last,Volume,Open,High,Low
2515,2011-06-10,$11.6393,433801306,$11.8054,$11.845,$11.6254
2514,2011-06-13,$11.6643,329376468,$11.6857,$11.7254,$11.6096
2513,2011-06-14,$11.8729,333995906,$11.7857,$11.9018,$11.7611
2512,2011-06-15,$11.6696,395841722,$11.7768,$11.7964,$11.6029
2511,2011-06-16,$11.6129,507299317,$11.675,$11.7386,$11.3689


In [34]:
apple_data.reset_index()

,index,Date,Close/Last,Volume,Open,High,Low
0,2515,2011-06-10,$11.6393,433801306,$11.8054,$11.845,$11.6254
1,2514,2011-06-13,$11.6643,329376468,$11.6857,$11.7254,$11.6096
2,2513,2011-06-14,$11.8729,333995906,$11.7857,$11.9018,$11.7611
3,2512,2011-06-15,$11.6696,395841722,$11.7768,$11.7964,$11.6029
4,2511,2011-06-16,$11.6129,507299317,$11.675,$11.7386,$11.3689
...,...,...,...,...,...,...,...
2511,4,2021-06-03,$123.54,76229170,$124.68,$124.85,$123.13
2512,3,2021-06-04,$125.89,75169340,$124.07,$126.16,$123.85
2513,2,2021-06-07,$125.9,71057550,$126.17,$126.32,$124.8321
2514,1,2021-06-08,$126.74,74403770,$126.6,$128.46,$126.2101


In [35]:
apple_data['Open'] = apple_data['Open'].apply(lambda x: x.replace('$',''))
apple_data['Open'] = apple_data['Open'].astype(float)

apple_data['Close/Last'] = apple_data['Close/Last'].apply(lambda x: x.replace('$',''))
apple_data['Close/Last'] = apple_data['Close/Last'].astype(float)

apple_data['High'] = apple_data['High'].apply(lambda x: x.replace('$',''))
apple_data['High'] = apple_data['High'].astype(float)

apple_data['Low'] = apple_data['Low'].apply(lambda x: x.replace('$',''))
apple_data['Low'] = apple_data['Low'].astype(float)

In [36]:
df = pd.merge(headlines, apple_data, how='inner', on=['Date'])
df

,Date,Headlines,Close/Last,Volume,Open,High,Low
0,2015-04-01,"Donald Yacktman on the Sources of Moats, His C...",31.0625,161852560,31.2050,31.2800,30.7750
1,2015-05-01,"SIM-free iPhone 6, 6 Plus reportedly debuting ...",32.2375,229396000,31.5250,32.5325,31.3250
2,2015-06-01,"Technology Transforming Cars Into ""Phones On W...",32.6337,128064080,32.8000,32.8475,32.5125
3,2015-07-01,Apple issues fourth developer beta of OS X 10....,31.6500,120827560,31.7250,31.7350,31.4975
4,2015-09-01,Samsung is reportedly making a secret new chip...,26.9300,306248680,27.5375,27.9700,26.8400
...,...,...,...,...,...,...,...
1590,2021-05-27,&quot;Friends: The Reunion&quot;: The Best Non...,125.2800,94625600,126.4400,127.6400,125.0800
1591,2021-05-28,Apple pushes back launch of podcast subscripti...,124.6100,71311110,125.5700,125.8000,124.5500
1592,2021-01-06,Judge dismisses charges against Apple security...,126.6000,155088000,127.7200,131.0499,126.3820
1593,2021-04-06,"Global equities nearly grasp all-time record, ...",126.2100,80171250,126.5000,127.1300,125.6500


In [37]:
df.set_index('Date', inplace = True)
df = df.sort_index(ascending=True, axis=0)
df

,Headlines,Close/Last,Volume,Open,High,Low
Date,,,,,,
2015-01-02,"Apple Loop: iOS 8.1.3 Angers Users, Outlook Ar...",27.3325,212575080,27.8475,27.8600,26.8375
2015-01-05,"Apple Watch interest described as &quot;tepid,...",26.5625,256843520,27.0725,27.1625,26.3525
2015-01-06,Apple&quot;s TV Service Delayed: Getting Local...,26.5650,262729000,26.6350,26.8575,26.1575
2015-01-07,Apple Inc. leaks new iPod colors in iTunes 12....,26.9375,159933400,26.8000,27.0500,26.6737
2015-01-08,Nomura Begins Coverage on Apple (AAPL) IBM cou...,27.9725,236675040,27.3075,28.0375,27.1750
...,...,...,...,...,...,...
2021-05-28,Apple pushes back launch of podcast subscripti...,124.6100,71311110,125.5700,125.8000,124.5500
2021-06-01,"TSX nears record high as cannabis, renewables ...",124.2800,67637120,125.0800,125.3500,123.9400
2021-06-02,Exclusive Social Media App Clubhouse Rises In ...,125.0600,59278860,124.2800,125.2400,124.0500
